In [2]:
import os
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [11]:
# 1. 프로젝트 폴더 기준 경로
# =============================================
FILES = {
    "역전": r"F:\데이터분석\역전할머니_정제완료.csv",
    "금별": r"F:\데이터분석\금별맥주_정제완료.csv"
}

OUTPUT_DIR = r"F:\데이터분석\출력"
os.makedirs(OUTPUT_DIR, exist_ok=True)

OUTPUT_FILES = {
    "역전": os.path.join(OUTPUT_DIR, "역전_블로그리뷰수.csv"),
    "금별": os.path.join(OUTPUT_DIR, "금별_블로그리뷰수.csv")
}

In [14]:
# 2. Selenium 드라이버 설정
# =============================================
options = Options()
# options.add_argument("--headless")  # 테스트 시 주석 해제 가능
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

In [15]:
# 3. 블로그 리뷰 수 크롤링 함수 (iframe 포함)
# =============================================
def get_blog_review_count(place_id: str) -> int:
    url = f"https://pcmap.place.naver.com/place/{place_id}"
    driver.get(url)
    time.sleep(1)  # 페이지 로딩 대기

    try:
        # 블로그 리뷰 탭 클릭
        blog_tab = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), '블로그 리뷰')]"))
        )
        blog_tab.click()
        time.sleep(1)

        # iframe 존재 시 전환
        try:
            iframe = driver.find_element(By.CSS_SELECTOR, "iframe#iframe")
            driver.switch_to.frame(iframe)
        except NoSuchElementException:
            pass  # iframe 없으면 그냥 진행

        # 리뷰 수 추출
        count_elem = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "em.place_section_count"))
        )
        count = int(count_elem.text.replace(",", ""))
        driver.switch_to.default_content()
        return count

    except (TimeoutException, NoSuchElementException):
        driver.switch_to.default_content()
        return 0

In [16]:
# 4. CSV 불러오기
# =============================================
def load_existing_data(input_csv: str, output_csv: str):
    df = pd.read_csv(input_csv, dtype={"place_id": str})
    if os.path.exists(output_csv):
        df_done = pd.read_csv(output_csv, dtype={"place_id": str})
        done_ids = set(df_done["place_id"])
        print(f"이미 크롤링 완료된 place_id: {len(done_ids)}")
    else:
        df_done = pd.DataFrame(columns=["place_id", "블로그_리뷰수"])
        done_ids = set()
        print("기존 크롤링 결과 없음 → 전체 크롤링")
    return df, df_done, done_ids

In [18]:
# 5. place_id 리스트 크롤링 + 체크포인트 저장
# =============================================
def crawl_place_ids(file_label: str, place_ids: list, checkpoint_csv: str = None):
    results = []
    total = len(place_ids)
    for idx, pid in enumerate(place_ids, 1):
        pid = str(int(float(pid)))  # 소수점 제거
        try:
            cnt = get_blog_review_count(pid)
            print(f"[{file_label}] [{idx}/{total}] place_id={pid} → 블로그 리뷰 수: {cnt}")
            results.append({"place_id": pid, "블로그_리뷰수": cnt})
        except Exception as e:
            print(f"[{file_label}] [{idx}/{total}] place_id={pid} → ❌ 오류: {e}")
            results.append({"place_id": pid, "블로그_리뷰수": 0})

        # 5개 단위 체크포인트 저장
        if checkpoint_csv and idx % 5 == 0:
            pd.DataFrame(results).to_csv(checkpoint_csv, index=False, encoding="utf-8-sig")
            print(f"[{file_label}] 💾 체크포인트 저장 ({idx}/{total})")

        time.sleep(0.8)
    return pd.DataFrame(results)

In [21]:
# 6. 전체 블로그 리뷰 크롤링 흐름
# =============================================
def crawl_blog_reviews(file_label: str, input_csv: str, output_csv: str):
    checkpoint_file = os.path.join(OUTPUT_DIR, f"{file_label}_checkpoint.csv")

    df, df_done, done_ids = load_existing_data(input_csv, output_csv)

    # 신규 크롤링 대상
    df_todo = df[~df["place_id"].isin(done_ids)].copy()
    print(f"[{file_label}] 이번에 크롤링할 대상: {len(df_todo)}")

    # 1차 크롤링
    df_new = crawl_place_ids(file_label, df_todo["place_id"].tolist(), checkpoint_csv=checkpoint_file)

    # 기존 데이터와 병합
    df_final = pd.concat([df_done, df_new], ignore_index=True).drop_duplicates("place_id", keep="first")

    # 리뷰 수 0인 항목 재크롤링
    df_retry = df_final[df_final["블로그_리뷰수"] == 0].copy()
    if len(df_retry) > 0:
        print(f"[{file_label}] 리뷰 수 0인 place_id {len(df_retry)}개 재크롤링 시작")
        df_retry_new = crawl_place_ids(file_label + " 재크롤링", df_retry["place_id"].tolist())
        # 기존 데이터 업데이트
        df_final.set_index("place_id", inplace=True)
        df_retry_new.set_index("place_id", inplace=True)
        df_final.update(df_retry_new)
        df_final.reset_index(inplace=True)

    # 최종 CSV 저장
    df_final.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"[{file_label}] ✅ 최종 파일 저장 완료: {output_csv}")

    # 체크포인트 삭제
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)
        print(f"[{file_label}] 🗑 체크포인트 파일 삭제: {checkpoint_file}")

In [22]:
# 7. 금별맥주 & 역전할머니 순차 크롤링
for label, input_path in FILES.items():
    crawl_blog_reviews(label, input_path, OUTPUT_FILES[label])

driver.quit()
print("전체 크롤링 완료")

이미 크롤링 완료된 place_id: 327
[역전] 이번에 크롤링할 대상: 0
[역전] 리뷰 수 0인 place_id 2개 재크롤링 시작


KeyboardInterrupt: 